In [1]:
from typing import List, Tuple
from pathlib import Path
from multiprocess import Pool
from multiprocess.pool import ThreadPool
import joblib
from joblib import Parallel, delayed, parallel_backend



import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import plotly.express as px
import tqdm

import talib

from statsmodels.tsa.stattools import coint

In [2]:
from market_data import MarketData

# Parameters

In [3]:
US_STOCK = False
FOREX = True

In [4]:
# profile_path = 'dataset/us_stock/profile.parquet'
# output_path = 'analysis/cointegration.csv'

us_stock_path = 'dataset/us_stock/profile.parquet'
forex_profile_path = 'dataset/forex/forex.parquet'
output_path = 'analysis_forex/cointegration.csv'

# Load Data

In [5]:
market_data = MarketData('dataset')

## US Stock


In [6]:
us_stock_df = pd.read_parquet(us_stock_path)
us_stock_df

,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,cik,isin,cusip,exchange,exchangeShortName,industry,website,description,ceo,sector,country,fullTimeEmployees,phone,address,city,state,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,MMM,116.81,0.986735,9717346,64667652096,5.95,107.07-186.3,3.699997,3M Company,USD,0000066740,US88579Y1010,88579Y101,New York Stock Exchange,NYSE,Conglomerates,https://www.3m.com,3M Company operates as a diversified technolog...,Mr. Michael Roman,Industrials,US,95000,16517331474,"3M Center, Bldg. 220-13E-26A",Saint Paul,MINNESOTA,55144-1000,7.79771,124.6080,https://financialmodelingprep.com/image-stock/...,1946-01-14,False,False,True,False,False
1,AOS,50.13,1.208548,1045389,7742478336,1.12,46.58-86.74,1.370003,A. O. Smith Corporation,USD,0000091142,US8318652091,831865209,New York Stock Exchange,NYSE,Specialty Industrial Machinery,https://www.aosmith.com,A. O. Smith Corporation manufactures and marke...,Mr. Kevin Wheeler,Industrials,US,13700,14143594000,11270 W Park Pl,Milwaukee,WISCONSIN,53224-9508,2.72625,52.8563,https://financialmodelingprep.com/image-stock/...,1994-12-14,False,False,True,False,False
2,ABT,95.06,0.709285,5323529,166470959104,1.88,93.25-142.6,-0.150002,Abbott Laboratories,USD,0000001800,US0028241000,002824100,New York Stock Exchange,NYSE,Medical Devices,https://www.abbott.com,"Abbott Laboratories, together with its subsidi...",Mr. Robert Ford,Healthcare,US,113000,12246676100,100 Abbott Park Rd,ABBOTT PARK,ILLINOIS,60064-6400,5.48656,100.5470,https://financialmodelingprep.com/image-stock/...,1937-03-01,False,False,True,False,False
3,ABBV,147.06,0.647383,5405496,260016783360,5.64,107.75-175.91,4.130005,AbbVie Inc.,USD,0001551152,US00287Y1091,00287Y109,New York Stock Exchange,NYSE,Drug Manufacturers—General,https://www.abbvie.com,"AbbVie Inc. discovers, develops, manufactures,...",Mr. Richard Gonzalez,Healthcare,US,50000,18479327900,1 N Waukegan Rd,North Chicago,ILLINOIS,60064-6400,5.52139,152.5810,https://financialmodelingprep.com/image-stock/...,2013-01-02,False,False,True,False,False
4,ABMD,256.22,1.314296,271576,11647991808,0.00,219.85-379.3,3.710007,"Abiomed, Inc.",USD,0000815094,US0036541003,003654100,NASDAQ Global Select,NASDAQ,Medical Devices,https://www.abiomed.com,"Abiomed, Inc. engages in the research, develop...",Mr. Michael Minogue,Healthcare,US,2003,19787775410,22 CHERRY HILL DR,Danvers,MASSACHUSETTS,01923,2.89484,259.1150,https://financialmodelingprep.com/image-stock/...,1987-07-29,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,CRUS,67.80,1.006263,467129,3792759296,0.00,61.94-95.84,2.330002,"Cirrus Logic, Inc.",USD,0000772406,US1727551004,172755100,NASDAQ Global Select,NASDAQ,Semiconductors,https://www.cirrus.com,"Cirrus Logic, Inc., a fabless semiconductor co...",Dr. Jason Rhode,Technology,US,1591,15128514000,800 W 6th St,Austin,TEXAS,78701,6.25288,74.0529,https://financialmodelingprep.com/image-stock/...,1989-06-09,False,False,True,False,False
726,RPM,88.73,0.938835,606259,11454954496,1.62,74.56-101.48,2.110001,RPM International Inc.,USD,0000110621,US7496851038,749685103,New York Stock Exchange,NYSE,Specialty Chemicals,https://www.rpminc.com,"RPM International Inc. manufactures, markets, ...",Mr. Frank Sullivan,Basic Materials,US,16751,13302735090,"2628 Pearl Rd, PO Box 777",Medina,OHIO,44258,1.91352,90.6435,https://financialmodelingprep.com/image-stock/...,1998-01-09,False,False,True,False,False
727,OSI,9.80,0.000000,9907,281750016,0.00,9.63-10.0,0.000000,Osiris Acquisition Corp.,USD,0001832136,None,None,New York Stock Exchange,NYSE,Shell Companies,https://osirisacquisitioncorp.com,Osiris Acquisition Corp. does not have signifi...,Mr. Benjamin E. Black,Financial Services,US,,646 993 4635,95 5th Avenue,New York,NY,10003,NaN,0.0000,https://financialmodelingprep.com/image-stock/...

## Forex

In [45]:
target_currencies = [
    'EUR', 'GBP', 'CAD', 'AUD', 'NZD', 'JPY',
    'CHF', 'SEK', 'NOK', 'MXN', 'ZAR', 'DKK',
    'INR', 'CNY', 'HKD', 'SGD', 
    'XAG', 'XAU', 'XPG'
]

In [46]:
forex_df = pd.read_parquet(forex_profile_path)
forex_df[['base', 'quote']] = forex_df['symbol'].map(lambda symbol: (symbol[:3], symbol[3:])).tolist()
forex_df = forex_df[(forex_df['base'] == 'USD') | (forex_df['quote'] == 'USD')]
forex_df = forex_df[forex_df['quote'].isin(target_currencies) | forex_df['base'].isin(target_currencies)]
forex_df

,symbol,name,currency,stockExchange,exchangeShortName,base,quote
90,USDNZD,USD/NZD,NZD,CCY,FOREX,USD,NZD
91,USDJPY,USD/JPY,JPY,CCY,FOREX,USD,JPY
92,HKDUSD,HKD/USD,USD,CCY,FOREX,HKD,USD
95,USDHKD,USD/HKD,HKD,CCY,FOREX,USD,HKD
97,USDGBP,USD/GBP,GBP,CCY,FOREX,USD,GBP
98,USDEUR,USD/EUR,EUR,CCY,FOREX,USD,EUR
100,USDCHF,USD/CHF,CHF,CCY,FOREX,USD,CHF
101,USDCAD,USD/CAD,CAD,CCY,FOREX,USD,CAD
102,USDAUD,USD/AUD,AUD,CCY,FOREX,USD,AUD
105,JPYUSD,JPY/USD,USD,CCY,FOREX,JPY,USD


# Load Data

In [47]:
symbols = []
if US_STOCK:
    symbols += us_stock_df['symbol'].tolist()
if FOREX:
    symbols += forex_df['symbol'].tolist()

In [48]:
price_df = market_data.read_batch(symbols, timeframe='day')
price_df

100%|██████████| 34/34 [00:00<00:00, 146.37it/s]


,USDNZD,USDJPY,HKDUSD,USDHKD,USDGBP,USDEUR,USDCHF,USDCAD,USDAUD,JPYUSD,SGDUSD,GBPUSD,NZDUSD,EURUSD,CADUSD,USDMXN,CHFUSD,USDCNY,USDZAR,ZARUSD,XAGUSD,USDINR,USDNOK,MXNUSD,XAUUSD,INRUSD,CNYUSD,USDDKK,DKKUSD,SEKUSD,NOKUSD,USDSEK,USDSGD,AUDUSD
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-09-24,NaN,NaN,0.129020,NaN,NaN,NaN,1.029070,1.09144,1.151700,0.011040,0.705570,1.59985,0.71813,1.46834,0.916300,NaN,0.971550,NaN,NaN,NaN,NaN,NaN,NaN,0.074050,NaN,0.020800,NaN,NaN,0.197230,NaN,0.171880,NaN,NaN,0.86789
2009-09-25,NaN,NaN,0.129020,NaN,NaN,NaN,1.029260,1.09134,1.151800,0.011150,0.705920,1.59317,0.71701,1.46684,0.915800,NaN,0.973440,NaN,NaN,NaN,NaN,NaN,NaN,0.073690,NaN,0.020850,NaN,NaN,0.197390,NaN,0.172240,NaN,NaN,0.86505
2009-09-27,1.400600,89.550,0.129020,7.74949,0.629900,NaN,1.034110,1.09828,1.159300,0.011160,0.704180,1.58652,0.71344,1.46073,0.910400,13.53399,0.966680,NaN,7.46229,0.133900,NaN,47.8630,5.84869,0.073590,987.60,0.020890,NaN,5.09399,0.196230,NaN,0.170640,7.02620,1.41950,0.86202
2009-09-28,1.389900,89.705,0.129030,7.74929,0.630600,NaN,1.035810,1.08331,1.144500,0.011150,0.704870,1.58598,0.71868,1.45773,0.922300,13.55069,0.965120,NaN,7.39099,0.135140,NaN,48.0610,5.84063,0.073750,991.95,0.020800,NaN,5.10559,0.195740,NaN,0.171080,7.02419,1.41790,0.87339
2009-09-29,1.390400,89.740,0.129020,7.74929,0.622300,NaN,1.033790,1.07987,1.136900,0.011140,0.706910,1.60667,0.71866,1.46010,0.925800,13.49719,0.967220,NaN,7.38149,0.135140,NaN,47.7180,5.83579,0.074060,995.50,0.020950,NaN,5.09749,0.196190,NaN,0.171290,7.00459,1.41390,0.87919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-19,1.776990,149.909,0.127402,7.84870,0.892710,1.02440,1.005700,1.38017,1.603900,0.006670,0.700633,1.12010,0.56240,0.97611,0.724480,20.15284,0.993729,7.2388,18.34150,0.054432,18.3082,83.0230,10.63783,0.049608,1626.20,0.012043,0.138106,7.62027,0.131182,0.088973,0.093947,11.23370,1.42713,0.62342
2022-10-20,1.768000,150.356,0.127393,7.84939,0.892810,1.02365,1.005780,1.37833,1.595810,0.006650,0.700977,1.11997,0.56550,0.97686,0.725440,20.03860,0.994150,7.2458,18.31490,0.054422,18.5192,82.7938,10.67975,0.049859,1621.54,0.012076,0.137973,7.61476,0.131315,0.088512,0.093568,11.28822,1.42643,0.62657
2022-10-21,1.726579,147.650,0.127379,7.84950,0.884486,1.01390,0.997511,1.35565,1.538580,0.006771,0.706364,1.13010,0.56880,0.98497,0.729102,19.90650,1.001890,7.2427,18.07399,0.053116,19.4062,82.5135,10.25574,0.050191,1656.60,0.012111,0.137859,7.54160,0.132584,0.089267,0.088582,11.15896,1.41470,0.62173


## Drop NaN symbol

In [49]:
missings = price_df.apply(lambda x: x[x.first_valid_index(): x.last_valid_index()].isna().sum(), axis=0)


In [50]:
first_open_dates = price_df.apply(lambda price: price.first_valid_index(), axis=0)
first_open_dates

USDNZD   2009-09-27
USDJPY   2009-09-27
HKDUSD   2009-09-24
USDHKD   2009-09-27
USDGBP   2009-09-27
USDEUR   2012-01-29
USDCHF   2009-09-24
USDCAD   2009-09-24
USDAUD   2009-09-24
JPYUSD   2009-09-24
SGDUSD   2009-09-24
GBPUSD   2009-09-24
NZDUSD   2009-09-24
EURUSD   2009-09-24
CADUSD   2009-09-24
USDMXN   2009-09-27
CHFUSD   2009-09-24
USDCNY   2011-07-11
USDZAR   2009-09-27
ZARUSD   2009-09-27
XAGUSD   2013-07-14
USDINR   2009-09-27
USDNOK   2009-09-27
MXNUSD   2009-09-24
XAUUSD   2009-09-27
INRUSD   2009-09-24
CNYUSD   2009-12-27
USDDKK   2009-09-27
DKKUSD   2009-09-24
SEKUSD   2010-09-08
NOKUSD   2009-09-24
USDSEK   2009-09-27
USDSGD   2009-09-27
AUDUSD   2009-09-24
dtype: datetime64[ns]

In [51]:
price_df = price_df[price_df.columns[first_open_dates < '2017-01-01']]

# Cointegration

In [52]:
def create_non_repeated_pair(symbols: List) -> List[tuple]:
    n_asset = len(symbols)
    pairs = []
    for i in range(n_asset):
        for j in range(i+1, n_asset):
            left_asset = symbols[i]
            right_asset = symbols[j]
            pairs.append((left_asset, right_asset))
    return pairs
            

In [53]:
pairs = create_non_repeated_pair(price_df.columns)


In [54]:
first_valid_indexes = price_df.apply(lambda prices: prices.first_valid_index())
last_valid_indexes = price_df.apply(lambda prices: prices.last_valid_index())

In [55]:
log_price_df = np.log(price_df)

In [56]:
def calculate_cointegration(symbols: tuple) -> dict:
    try:
        left_symbol, right_symbol = symbols
        # left_first_date = first_valid_indexes[left_symbol]
        # right_first_date = first_valid_indexes[right_symbol]

        # start_date = max(left_first_date, right_first_date)

        # left_last_date = last_valid_indexes[left_symbol]
        # right_last_date = last_valid_indexes[right_symbol]

        # end_date = min(left_last_date, right_last_date)

        # left_prices = log_price_df.loc[start_date: end_date, left_symbol]
        # right_prices = log_price_df.loc[start_date: end_date, right_symbol]

        valid_dates = log_price_df[left_symbol].notna() & log_price_df[right_symbol].notna()
        left_prices = log_price_df.loc[valid_dates, left_symbol]
        right_prices = log_price_df.loc[valid_dates, right_symbol]

        t_score, pvalue, _ = coint(left_prices, right_prices)

        return {
            'left_symbol': left_symbol,
            'right_symbol': right_symbol,
            'cointegration': 1 - pvalue,
            'cointegration_test_period': len(left_prices)
        }
    except Exception as e:
        print(e)
        
        return {
            'left_symbol': left_symbol,
            'right_symbol': right_symbol,
            'cointegration': None,
            'cointegration_test_period': 0
        }
# results = list(tqdm.tqdm(map(calculate_cointegration, pairs), total=len(pairs)))

In [57]:


results = Parallel(n_jobs=-1)(delayed(calculate_cointegration)(symbols) for symbols in tqdm.tqdm(pairs))
cointegration_df = pd.DataFrame(results)


100%|██████████| 561/561 [00:23<00:00, 23.65it/s]


In [58]:
cointegration_df.to_csv(output_path, index=False)

In [59]:
cointegration_df

,left_symbol,right_symbol,cointegration,cointegration_test_period
0,USDNZD,USDJPY,0.785850,4110
1,USDNZD,HKDUSD,0.505716,4053
2,USDNZD,USDHKD,0.466705,4057
3,USDNZD,USDGBP,0.922822,4062
4,USDNZD,USDEUR,0.920139,3320
...,...,...,...,...
556,NOKUSD,USDSGD,0.921989,3979
557,NOKUSD,AUDUSD,0.989823,4076
558,USDSEK,USDSGD,0.825440,3936
559,USDSEK,AUDUSD,0.827390,3912
